# CSC421 Assignment 2 - Part I  Propositional Logic (5 points) #
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 7 - Logical Agents** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the code provided in logic.py and logic.ipynb for completing the assignment questions. This part does NOT rely on the provided code so you can complete it just using basic Python. 


```
Birds can fly, unless they are penguins and ostriches, or if they happen 
to be dead, or have broken wings, or are confined to cages, or have their 
feet stuck in cement, or have undergone experiences so dreadful as to render 
them psychologically incapable of flight 

Marvin Minsky 
```

# Introduction - Parsing and evaluating prefix logic expressions  

In this assignment your task is to incrementally create a parser and evaluator for prefix logic expressions as well as implement simple model checking. **NOTE THAT THE GRADING IN THIS ASSIGNMENT IS DIFFERENT FOR GRADUATE STUDENTS AND THEY HAVE TO DO EXTRA WORK FOR FULL MARKS**


# Question 1A (Minimum) CSC421 -  (1 point, CSC581C - 0 points) 

Your first task will be to write a simple evaluator of prefix logic expressions with constants. In prefix notation the operator precedes the operands and no operands are required. For example 5+3 in prefix notation is written + 5 3 or 5 * 2 + 3 would be written + * 5 2 3 or + * 5 2 * 3 4 is equivalent to (5 * 2) + (3 * 4). 

As a first step we will consider very simple expressions with one operator and two constant operands. We will use 0 for false and 1 for true. The following logical connectives should be implemented (see Figure 7.8 in your book) (notice that for now there is no negation symbol ~): 

1. &    (and), 
3. |    (or), 
4. =>   (implication) 
5. <=>  (biconditional) 

Example expressions: 
```
& 1 0  
=> 0 1 
<=> 1 1 
```

Your function should take as input a string with the prefix expression and return the result of evaluating the expression (basically a 1 for true and 0 for false). You can split a string to a list using .split[' ']. For this part of the assignment you only evaluate expressions with two constant operands i.e no nested/recursive expressions. 

In [9]:
def IMPLICATION(a, b):
    return int(not a or b)
def BICONDITIONAL(a, b):
    return int(IMPLICATION(a, b) and IMPLICATION(b, a))

def eval_logic(s):
    x = s.split(' ')
    a, b = int(x[1]), int(x[2])
    if x[0] == '&':
        return a and b
    if x[0] == '|':
        return a or b
    if x[0] == '=>':
        return IMPLICATION(a, b)
    if x[0] == '<=>':
        return BICONDITIONAL(a, b)
    return -1

print(eval_logic('& 1 0'))
print(eval_logic('| 1 0'))
print(eval_logic('=> 1 0'))
print(eval_logic('<=> 0 0'))
        

0
1
0
1


# Question 1B (Minimum) (CSC421 - 1 point, CSC581C - 0 point) 

Your next task is to implement variables and bindings for your propositional logic evaluator. In this version in addition to constants (0 and 1) you also can have variables which are strings with associated values provided in a dictionary. You still only consider two operands and one operator (no nesting). For example in the code below 
the three expressions are equivalent. Your function should take as arguments the expression to be evaluated as a string and the dictionary with the variable bindings. In addition you need to add the ~ (not) operator. To do so for each variable in the dictionary add a not version. For example if 'a' in the dictionary has a value of 1 the '~a' in the dictionary should have a value of 0. Notice that the not symbol is part of the string and NOT separated by a space. 



In [53]:
d = {'foo': 0, 'b': 1}
print(d)
expr1 = '& 0 1'
expr2 = '& foo 1'
expr3 = '& foo ~b'

{'foo': 0, 'b': 1}


In [22]:
def get_value(s, d):
    try:
        return d[s]
    except KeyError:
        pass
    try:
        return int(not int(s[1:])) if s.startswith('~') else int(s)
    except ValueError:
        print('Value Error')
        return -1
    
def eval_logic(s, d={}):
    x = s.split(' ')
    a = get_value(x[1], d)
    b = get_value(x[2], d)
    if x[0] == '&':
        return a and b
    if x[0] == '|':
        return a or b
    if x[0] == '=>':
        return IMPLICATION(a, b)
    if x[0] == '<=>':
        return BICONDITIONAL(a, b)
    return -1
    
d = {'a': 1, '~a': 0, 'b': 0, '~b': 1}
print(eval_logic('| a b', d))
print(eval_logic('& a ~b', d))
print(eval_logic('=> ~a ~b', d))
print(eval_logic('<=> a b', d))

1
1
1
0


# Question 1C (Expected) 1 point 


The following code is a recursive evaluator for prefix arithmetic expressions. It assumes that there are always two operands either an integer or a prefix expression starting with an operator (addition or multiplication). It is a good idea to go through this function carefully by hand to understand how the recursion works. 

Informed by your understanding of the arithmetic recursive_eval function your task is to write function to implement a recursive prefix logic evaluator. Your evaluator should also support variables bindings using a dictionary as in the previous question. 

Example expressions: 
```
& 1 & 1 a   
=> 0 & b ~alice  
<=> foo 1 
```

In [1]:
def recursive_eval(l):
    head, tail = l[0], l[1:]
    if head in ['+', '*']: 
        val1, tail = recursive_eval(tail)
        val2, tail = recursive_eval(tail)
        if head == '+': 
            return (int(val1)+int(val2), tail)
        elif head == '*':  
            return (int(val1)*int(val2), tail)
    # operator is a number 
    else:  
        return (int(head),tail)

def prefix_eval(input_str): 
    input_list = input_str.split(' ')
    res, tail = recursive_eval(input_list)
    return res

print(prefix_eval('1'))
print(prefix_eval('+ 1 2'))
print(prefix_eval('+ 1 * 2 3'))
print(prefix_eval('+ * 5 2 * 3 + 1 5'))

1
3
7
28


In [33]:
def recursive_eval_logic(s, d):
    head, tail = s[0], s[1:]
    if head in ['&', '|', '=>', '<=>']:
        val1, tail = recursive_eval_logic(tail, d)
        val2, tail = recursive_eval_logic(tail, d)
        return (eval_logic(' '.join([head, str(val1), str(val2)])), tail)
    else:
        return (get_value(head, d), tail)
    
def recursive_logic(s, d={}):
    res, tail = recursive_eval_logic(s.split(' '), d)
    return res
        
print(recursive_logic('& 1 | 1 0'))
print(recursive_logic('& 1 & 1 a', d))
print(recursive_logic('& 1 & 1 ~a', d))
print(recursive_logic('<=> b 1', d))

1
1
0
0


# QUESTION 1D (EXPECTED) 1 point


Using the recursive prefix evaluator you defined in the previous question 
answer the following question (you will need to convert the exressions below 
to prefix). You can use multiple string assignments to assemble more complicated 
sentences into one big string: 


Let A be the formula: 

\begin{equation} 
  (( p_{1} \rightarrow (p2 \land p_{3})) \land ((\neg p_{1})
  \rightarrow (p_{3} \land p_{4})))
\end{equation} 

Let B be the formula: 

\begin{equation} 
  (( p_{3} \rightarrow (\neg p_{6})) \land ((\neg
  p_{3}) \rightarrow (p_{4} \rightarrow p_{1})))  
\end{equation} 

Let C be the formula: 

\begin{equation} 
  ((\neg(p2 \land p_{5})) \land (p2 \rightarrow p_{5})) 
\end{equation} 

Let D be the formula: 

\begin{equation} 
  (\neg (p_{3} \rightarrow p_{6}))
\end{equation} 

Evaluate the formulate E: 
\begin{equation} 
  (( A \land (B \land C)) \rightarrow D)
\end{equation} 

under the true assignment $I_{1}$, where $I_{1}(p_{1}) = I_{1}(p_{3}) = I_{1}(p_{5}) = false$ 
and $I_{1}(p2) = I_{1}(p_{4}) = I_{1}(p_{6}) = true$ as well as under the truth assignment 
$I_{2}$, where $I_{2}(p_{1}) = I_{2}(p_{3}) = I_{2}(p_{5}) = true$ and
$I_{2}(p_{2})=I_{2}(p_{4})=I_{2}(p_{6}) = false$. 


In [57]:
# YOUR CODE GOES HERE

# QUESTION 1E (ADVANCED) 1 point 

Implement inference using model-checking using your prefix recursive evaluator to decide whether a knowledge base KB entais some sentence a. To do so express the knowledge base in the prefix notation, enumerate all models for the variables in the dictionary, and check that the sentence a is true in every model in which the KB is true. 

You can check the implementation to tt_entails in logic.ipynb in the aima_python repository to inform how you implement your solution. Your solution should NOT rely directly on any code in logic.py or logic.ipynb. 

Check your model checking using the examples that are used in logic.ipynb to check entailment (there are a few with P and Q as variables as well as the one with A, B, C, D, E, F, G. You will need to convert these examples to prefix notation. 


In [58]:
# YOUR CODE GOES HERE 


# QUESTION 1F (ADVANCED) (CSC421 - 0 points, CSC581C - 1 point)

Implement conversion of the prefix expressions to prefix conjuctive normal form (CNF) based on the recursive evaluator you have implemented. 

In [50]:
# YOUR CODE GOES HERE 

# QUESTION 1E (ADVANCED) (CSC421 - 0 POINTS, CSC581C 1 point)

Based on the recursive evaluator you have implemented do a conversion of expressions in prefix notation to the infix notation of expressions supported by logic.ipynb. Provide 4 test cases that demonstrate the the conversion works by confirming that the result of your evaluator and the logic.ipynb evaluator are the same. 

In [ ]:
# YOUR CODE GOES HERE 